### Imports 

In [ ]:
import os

os.chdir('../')

# HTML PARSER
from bs4 import BeautifulSoup as soup

# INITIALIZE DRIVER
from selenium import webdriver

# WAIT AND CONDITIONS METHODS
# available since 2.26.0
from selenium.webdriver.support.ui import Select

# Exceptions
from selenium.common.exceptions import TimeoutException


# METHODS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from locators import Login, Base, LatMenu, \
    Main, ListaBlocos, Bloco, Processo, Envio


from base import Page

import re

import pandas as pd

In [ ]:
def login_SEI(driver, usr, pwd):
        """
        Esta função recebe um objeto Webdrive e as credenciais  
        do usuário, loga no SEI - ANATEL e retorna uma instância da classe  
        SEI. 
        """

        page = Page(driver)
        page.driver.get(Login.URL)
        page.driver.maximize_window()

        usuario = page.wait_for_element_to_click(Login.LOGIN)
        senha = page.wait_for_element_to_click(Login.SENHA)

        # Clear any clutter on the form
        usuario.clear()
        usuario.send_keys(usr)

        senha.clear()
        senha.send_keys(pwd)

        # Hit Enter
        senha.send_keys(Keys.RETURN)

        return SEI(page.driver)

In [ ]:
class SEI(Page):
    """
    Esta subclasse da classe Page define métodos de execução de ações na 
    página principal do SEI e de resgate de informações
    """
    
    processos = []
    
    def ver_proc_detalhado(self):
        """
        Expands the visualization from the main page in SEI
        """
        try:
            ver_todos = self.wait_for_element_to_click(Main.FILTROATRIBUICAO)
            
            if ver_todos.text =="Ver todos os processos":
                ver_todos.click()
        
        except TimeoutException:
            
            print("A página não carregou no tempo limite ou cheque o link\
                  'ver todos os processos'")
            
        try:
            
            visual_detalhado = self.wait_for_element_to_click(Main.TIPOVISUALIZACAO)
        
            if visual_detalhado.text == "Visualização detalhada":
                visual_detalhado.click()
                
        except TimeoutException:
            
            print("A página não carregou no tempo limite ou cheque o link\
            de visualização detalhada")
            
    def isPaginaInicial(self):
        """Retorna True se a página estiver na página inicial do SEI, False
        caso contrário"""        
        return self.get_title() == 'SEI - Controle de Processos'

    def go_to_initial_page(self):
        """
        Navega até a página inicial do SEI caso já esteja nela 
        a página é recarregada
        Assume que o link está presente em qualquer subpágina do SEI
        """
        self.wait_for_element_to_click(
            Base.INITIALPAGE).click()

    def exibir_menu_lateral(self):
        """
        Exibe o Menu Lateral á Esquerda no SEI para acessos aos seus diversos
        links
        Assume que o link está presente em qualquer subpágina do SEI
        """
        menu = self.wait_for_element(Base.EXIBIRMENU)

        if menu.get_attribute("title") == "Exibir Menu do Sistema":
            menu.click()
            
    def itera_processos(self):
        """
        Navega as páginas de processos abertos no SEI e guarda as tags
        html dos processos como objeto soup no atributo processos_abertos
        """     
        
        #Apaga o conteúdo atual da lista de processos
        self.processos = []
        
        # assegura que está inicial
        if not self.isPaginaInicial():
            self.go_to_initial_page()

        # Mostra página com informações detalhadas
        self.ver_proc_detalhado()
        
        contador = Select(self.wait_for_element(Main.CONTADOR))

        pages = [pag.text for pag in contador.options]

        for pag in pages:
            
            # One simple repetition to avoid more complex code
            contador = Select(self.wait_for_element(Main.CONTADOR))
            contador.select_by_visible_text(pag)
            html_sei = soup(self.driver.page_source, "lxml")
            self.processos += html_sei("tr", {"class": 'infraTrClara'})
            break
            
        # percorre a lista de processos
        # cada linha corresponde a uma tag mãe 'tr'
        # substituimos a tag mãe por uma lista das tags filhas 'tag.contents', descartando os '\n'
        # a função lista_to_dict_tags recebe essa lista e retorna um dicionário das tags
        self.processos =  [lista_to_dict_tags([tag for tag in line.contents if tag !='\n']) for line in self.processos]     

In [126]:
def lista_to_dict_tags(lista_tags):

    tags = ['checkbox', 'anotacao', 'prioridade', 'situacao', 'marcador', 'aviso', 
            'peticionamento', 'processo', 'atribuicao', 'tipo', 'interessado']
    
    dict_tags = {tag:None for tag in tags}

    assert len(lista_tags) == 6, "Verifique o nº de tags de cada linha do processo, valor diferente de 10"

    dict_tags['checkbox'] = lista_tags[0].find(class_='infraCheckbox')

    controles = lista_tags[1].find_all('a')
    
    for tag_a in controles:
        
        img = str(tag_a.img['src'])
        
        if 'imagens/sei_anotacao' in img:
            
            dict_tags['anotacao'] = tag_a
            
            if 'prioridade' in img: 
                
                dict_tags['prioridade'] = True
                
            else:                
                
                dict_tags['prioridade'] = False
            
        elif 'imagens/sei_situacao' in  img:
            
            dict_tags['situacao'] = tag_a
            
        elif 'imagens/marcador' in img:
            
            dict_tags['marcador'] = tag_a
            
        aviso = lista_tags[1].find(src = re.compile('imagens/exclamacao'))
        
        if aviso:
            
            dict_tags['aviso'] = re.search('\((.*)\)', aviso.attrs['onmouseover']).group()
            
        peticionamento = lista_tags[1].find(src = re.compile('peticionamento'))
        
        if peticionamento:
            
            dict_tags['peticionamento'] = re.search('\((.*)\)', peticionamento.attrs['onmouseover']).group()
            
    try:    
        dict_tags['processo'] = lista_tags[2].find('a')       
    except:        
        print("Problema na tag 'processo'")
        
    try:
        dict_tags['atribuicao'] = lista_tags[3].find('a').string
    except:
        print("Problema na tag ou processo sem atribuição")
        
    try:    
        dict_tags['tipo'] = lista_tags[4].string
    except:
        print("Problema na tag 'tipo de processo'")
    
    try:        
        dict_tags['interessado'] = lista_tags[5].find(class_='spanItemCelula').string        
    except:        
        print("Processo: " + dict_tags['processo'].string + " Problema na tag ou processo sem 'interessado'")

    return dict_tags
    
    


In [ ]:
def dict_to_df(processos):
    
     tags = ['anotacao', 'prioridade', 'situacao', 'marcador', 'aviso', 
             'processo', 'atribuicao', 'tipo', 'interessado']
    
    atributos = {tag:None for tag in tags}
    
    for num_proc, dict_tags in processos.items():
        
        anotacao = dict_tags['anotacao']
        
        if anotacao:
            
            atributos['anotacao'] = re.search('\(.*)\)', anotacao.attrs['onmouseover']).group()
            
            if 'prioridade' in str(anotacao.img):
                
                atributos['prioridade'] = True
                
        situacao = dict_tags['situacao']
        
        if situacao:
            
            atributos['situacao'] = re.search('\(.*)\)', situacao.attrs['onmouseover']).group()
            
        marcador = dict_tags['marcador']
        
        if marcador:
            
            atributos['marcador'] = re.search('\(.*)\)', marcador.attrs['onmouseover']).group()
            
        aviso = dict_tags['aviso']
        
        if aviso:
            
            atributos['aviso'] = re.search('\(.*)\)', aviso.attrs['onmouseover']).group()
            
        
            
            
        
            
        
            
        
            
    
    
        



In [109]:
driver = webdriver.Chrome()
sei = login_SEI(driver, 'rsilva', 'Savorthemom3nts')

In [127]:
sei.itera_processos()
sei.processos[6]

Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição
Problema na tag ou processo sem atribuição


AttributeError: 'NoneType' object has no attribute 'group'

In [90]:
for p in sei.processos:
    
    for v in p.values():
    
        if v['aviso']:
        
            print(v['aviso'])

<a href="javascript:void(0);" onmouseout="return infraTooltipOcultar();" onmouseover="return infraTooltipMostrar('Um documento foi incluído ou assinado neste processo');"><img class="imagemStatus" src="imagens/exclamacao.png"/></a>
<a href="javascript:void(0);" onmouseout="return infraTooltipOcultar();" onmouseover="return infraTooltipMostrar('Um documento foi incluído ou assinado neste processo');"><img class="imagemStatus" src="imagens/exclamacao.png"/></a>
<a href="javascript:void(0);" onmouseout="return infraTooltipOcultar();" onmouseover="return infraTooltipMostrar('Um documento foi incluído ou assinado neste processo');"><img class="imagemStatus" src="imagens/exclamacao.png"/></a>
<a href="javascript:void(0);" onmouseout="return infraTooltipOcultar();" onmouseover="return infraTooltipMostrar('Um documento foi incluído ou assinado neste processo');"><img class="imagemStatus" src="imagens/exclamacao.png"/></a>
<a href="javascript:void(0);" onmouseout="return infraTooltipOcultar();"

In [42]:
re.search('\(.+\)', text).group()

"('ANALISADO','rodrigoferraz')"

In [ ]:
sei.processos[10]['processo'].attrs['href']


In [ ]:
processos = lista

In [38]:
tag = <img src="modulos/peticionamento/imagens/peticionamento_processo_novo.png" onmouseout="return infraTooltipOcultar();" onmouseover="return infraTooltipMostrar(&quot;Processo Novo: 13/09/2017&quot;,&quot;Peticionamento Eletrônico&quot;);">

Help on function compile in module re:

compile(pattern, flags=0)
    Compile a regular expression pattern, returning a pattern object.

